<a href="https://colab.research.google.com/github/Didier06/IA_licence_pro_chimie/blob/main/VeilleChimie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install feedparser==6.0.11

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.6 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=ac70db3599b2c6db66116d4533a28b1d74ec7472d02b76b1d58209e4ddd95448
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k


In [2]:
import feedparser
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from datetime import date, datetime
import openai
import requests
import os
import urllib.parse

In [3]:
# CONFIGURATION
openai.api_key = "sk-proj-xxxxxxx"  # Remplace par ta vraie clé OpenAI
# pour obtenir un clé openai https://platform.openai.com/settings/organization/api-keys
GMAIL_USER = "votremail@gmail.com" # votre email
GMAIL_APP_PASSWORD = "xxxxxxxxxxx" # # pour créer un mot de passe application avec gmail : https://myaccount.google.com/apppasswords

DESTINATAIRES = ["destinataire1@gmail.com", "destinataire2@gmail.com"]
DOSSIER_SAUVEGARDE = "veille_sauvegardes_sensors"
LOG_FILE = "veille_log.txt"

In [4]:
# FONCTIONS

def get_arxiv_articles(query="arduino OR sensor chemical OR detection", max_results=5):
    encoded_query = urllib.parse.quote(query)
    url = f"http://export.arxiv.org/api/query?search_query=all:{encoded_query}&start=0&max_results={max_results}&sortBy=lastUpdatedDate"
    feed = feedparser.parse(url)
    return [(entry.title, entry.link, entry.summary) for entry in feed.entries]

def get_pubchem_trending():
    url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/fastidentity/listkey/JSON"
    try:
        r = requests.get(url, timeout=10)
        if r.status_code == 200:
            return ["Résultats PubChem à ajouter manuellement"]
    except Exception:
        pass
    return []

def get_cen_articles():
    rss_url = "https://cen.acs.org/content/cen/en/news.html.rss"
    feed = feedparser.parse(rss_url)
    return [(entry.title, entry.link, entry.summary) for entry in feed.entries[:5]]

def get_techcrunch_articles():
    rss_url = "https://techcrunch.com/tag/chemical/feed/"
    feed = feedparser.parse(rss_url)
    return [(entry.title, entry.link, entry.summary) for entry in feed.entries[:5]]

def get_maker_articles():
    rss_url = "https://makezine.com/category/electronics/feed/"
    feed = feedparser.parse(rss_url)
    return [(entry.title, entry.link, entry.summary) for entry in feed.entries[:5]]

def resume_article(title, summary):
    prompt = f"""Tu es un veilleur technologique pour un laboratoire de chimie.
Résume cet article en 3 phrases et précise s’il peut aider à détecter ou mesurer quelque chose en chimie avec un Arduino ou un capteur simple.
Titre : {title}
Résumé : {summary}"""
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.5,
            max_tokens=300
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Erreur GPT : {e}"


In [5]:
def build_html_report():
    html = f"<h2>Envoye par votre assistant IA : capteurs Arduino en Chimie - {date.today().strftime('%d %B %Y')}</h2>"

    sources = {
        "ArXiv (Capteurs Arduino & Chimie)": get_arxiv_articles(),
        "C&EN (Industrie chimique)": get_cen_articles(),
        "TechCrunch (Startups chimiques)": get_techcrunch_articles(),
        "PubChem (Données moléculaires)": [(t, "", "") for t in get_pubchem_trending()],
        "Makers / Capteurs Arduino": get_maker_articles()
    }

    for nom_source, articles in sources.items():
        html += f"<h3>{nom_source}</h3><ul>"
        for titre, lien, resume in articles:
            synthese = resume_article(titre, resume)
            html += f"<li><b>{titre}</b><br><a href='{lien}'>{lien}</a><br>{synthese}</li><br>"
        html += "</ul>"
    return html

def sauvegarder_html_local(html):
    os.makedirs(DOSSIER_SAUVEGARDE, exist_ok=True)
    nom_fichier = f"veille_{date.today().isoformat()}.html"
    chemin_complet = os.path.join(DOSSIER_SAUVEGARDE, nom_fichier)
    with open(chemin_complet, "w", encoding="utf-8") as f:
        f.write(html)
    return chemin_complet

def enregistrer_log(chemin_html):
    horodatage = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(f"[{horodatage}] Veille envoyée - Fichier : {chemin_html}\n")

def envoyer_email(html_content):
    msg = MIMEMultipart('alternative')
    msg['Subject'] = "[Veille Chimie] Rapport hebdo"
    msg['From'] = GMAIL_USER
    msg['To'] = ", ".join(DESTINATAIRES)

    part = MIMEText(html_content, 'html')
    msg.attach(part)

    server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    server.login(GMAIL_USER, GMAIL_APP_PASSWORD)
    server.sendmail(GMAIL_USER, DESTINATAIRES, msg.as_string())
    server.quit()

# MAIN
if __name__ == "__main__":
    contenu_html = build_html_report()
    chemin_html = sauvegarder_html_local(contenu_html)
    enregistrer_log(chemin_html)
    envoyer_email(contenu_html)
    print("Veille envoyée avec succès.")


Veille envoyée avec succès.
